<a href="https://it-omscholing.nl/locaties/hogeschool-rotterdam/">
<div>
<a><img src='../../../pics/banner.PNG'/></a>
</div>
<div>
<a href=''><img src='../../../pics/miw.PNG'/></a>
<em>Author: Jeroen Boogaard</em>
</div>
</a>

<h2>Voorbeeld image augmentation</h2>

<h3>Imports</h3>

In [76]:
from os import path, listdir
from torch.utils.data import Dataset
from torchvision.utils import save_image
import cv2
import torchvision.transforms as transforms
import pathlib
import matplotlib.pyplot as plt
%matplotlib inline

<h3>Global variabels</h3>

In [ ]:
forestDirectory = '../../../pics/2750/Forest'
residentialDirectory = '../../../pics/2750/Residential'

<h3>Data Collection</h3>
<p>
We gebruiken Images uit de <a href="https://www.kaggle.com/datasets/apollo2506/eurosat-dataset">EuroSat dataset</a> die gemaakt zijn met de Sentinel-2 sateliet. Elke image is een 64x64 pixels foto van Europees aardoppervlak op een hoogte van 10 meter. De images zijn te categoriseren in Highway, Industrial, Pasture, PermanentCrop, Residential, River en SeaLake.
</p>
<img src=../../../pics/eurosat_overview_small.jpg length=50% width=50%>
<p>
Download <a href="http://madm.dfki.de/files/sentinel/EuroSAT.zip">EuroSAT.zip</a> en kopieer daaruit de directory 2750 naar opdrachten/practica/pics.      
<strong>Voeg het pad naar de directory 2750 toe aan .gitignore zodat je de plaatjes niet naar je remote git repository pusht</strong>
</p>    

<h3>Data Selection</h3>

In [ ]:
landFiles = list()
landLabels = list()
    
for fileName in listdir(residentialDirectory):
    imgFile = path.join(residentialDirectory, fileName)

    if ".jpg" in imgFile:
        landFiles.append(imgFile)
        landLabels.append(fileName)

In [74]:
class LandDataset(Dataset):
    def __init__(self, imagePaths, transform=False):
        self.image_paths = imagePaths
        self.transform = transform
        
    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_filepath = self.image_paths[idx]
        image = cv2.imread(image_filepath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Use Pahtlib
        label = image_filepath.split('/')[-2]
        label = class_to_idx[label]
        if self.transform is not None:
            image = self.transform(image=image)["image"]
        
        return image, label

In [ ]:
landTransforms = transforms.Compose (
    [
        transforms.ToPILImage(),
        transforms.Resize( (256, 256) ),
        transforms.RandomCrop( (224, 224) ),
        transforms.ColorJitter(brightness=0.5),
        transforms.RandomRotation(degrees=45),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.05),
        transforms.ToTensor()
    ]    
)    

In [ ]:
landDataset = LandDataset(residentialDirectory, transform=landTransforms)

In [ ]:
print('The shape of tensor for 50th image in train dataset: ',landDataset[49][0].shape)
# print('The label for 50th image in train dataset: ',landDataset[49][1])

In [ ]:
for img, label in landDataset:
    augImgFile = "tes.jpg"
    save_image(img, augImgFile)

In [ ]:
def visualize_augmentations(dataset, idx=0, samples=10, cols=5, random_img = False):
    
    dataset = copy.deepcopy(dataset)
    #we remove the normalize and tensor conversion from our augmentation pipeline
    dataset.transform = A.Compose([t for t in dataset.transform if not isinstance(t, (A.Normalize, ToTensorV2))])
    rows = samples // cols
    
        
    figure, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(12, 8))
    for i in range(samples):
        if random_img:
            idx = np.random.randint(1,len(train_image_paths))
        image, lab = dataset[idx]
        ax.ravel()[i].imshow(image)
        ax.ravel()[i].set_axis_off()
        ax.ravel()[i].set_title(idx_to_class[lab])
    plt.tight_layout(pad=1)
    plt.show()    

visualize_augmentations(train_dataset,np.random.randint(1,len(train_image_paths)), random_img = True)

https://towardsdatascience.com/custom-dataset-in-pytorch-part-1-images-2df3152895